# Quickly and Easily Calculate Business Cycle Moments

### by [Kerk L. Phillips](https://sites.google.com/site/kerkphillips/home), July 2018

The code in this Jupyter notebook was written using Python 3.6.

This notebook shows how to quickly and easily calculate a set of commonly reported moments from a set of data.  This data could be real world data, or data from a simulated model.  The function used, `calcmom`, can be downloaded at the following Github site.  [https://github.com/kerkphil/DSGE-Utilities](https://github.com/kerkphil/DSGE-Utilities).  For the example in this notebook we use data stored in a csv spreadsheet, `DSGEtestdata.csv`.  That data files and this notebook can be found in the public GitHub repository [https://github.com/kerkphil/Jupyter_Notebooks](https://github.com/kerkphil/Jupyter_Notebooks).

## Moment Calculator Code

The code below can be downloaded from the repository shown above, however we reproduce it here in order to see how the moments are calculated.

Note that the data must be organized as follows:

data: an nobs-by-nvar matrix of data with with nobs indexing the time-series observations in rows and nvar indexing the variables in columns.  If GDP is included it should be in the first column.

In [2]:
import pandas as pd
import numpy as np

def calcmom(data, means = False, stds = True, relstds = True, 
            corrs = True, autos = True, cvars = False):
    '''
    This function calculates a user controlled set of moments from data
    It takes the following as inputs:
    1)  data: an nobs-by-nvar matrix of data with with nobs indexing the 
        time-series observations in rows and nvar indexing the variables in 
        columns.  If GDP is included it should be in the first column.
    
    The function caculates and returns the following outputs in a single
    matrix with  rows for each type of momemnt and nvar columns .  The moments,
    if calculated are reported in the following order.  Each column can be
    turned off or on by setting the appropriate flags.
    1) means -     means of the variables
    2) stdevs -    standard deviations of variables
    3) relstds -   standard deviations of variables relative to GDP
    4) correls -   correlations of variables with GDP
    5) autocoors - autocorrelations of variables
    6) coefvars -  coefficients of variation (stdev/mean)
        
    Notes:
   
    '''
    # find the number of variables and observations
    (nobs, nvar) = data.shape
    report = np.zeros(nvar)
    count = 0
    rindex = []
    
    if means or cvars:
        dmeans= np.mean(data,axis=0)
        report = np.vstack((report,dmeans))
        count = count + 1
        rindex.extend(['means'])
        
    if stds or cvars:
        dstds = np.std(data,axis=0)
        report = np.vstack((report,dstds))
        count = count + 1
        rindex.extend(['standard deviations'])
        
    if relstds:
        drelstds = dstds/dstds[0]
        report = np.vstack((report,drelstds))
        count = count + 1
        rindex.extend(['standard deviations relative to GDP'])
        
    if corrs:
        temp = np.corrcoef(np.transpose(data))
        dcorrs = temp[0:nvar,0]
        report = np.vstack((report,dcorrs))
        count = count + 1
        rindex.extend(['corrleations with GDP'])
        
    if autos:
        dlead = np.transpose(data[1:nobs,:])
        dlag = np.transpose(data[0:nobs-1,:])
        temp = np.corrcoef(dlead,dlag)
        dautos = np.diag(temp, -nvar)
        report = np.vstack((report,dautos))
        count = count + 1
        rindex.extend(['autocorrelations'])
        
    if cvars:
        dcvars = dstds/dmeans
        report = np.vstack((report,dcvars))
        count = count + 1
        rindex.extend(['coefficients of variation'])
        
    # appropriately truncate lists
    report = report[1:count+1,:]
    rindex = rindex[0:count+1]
    
    return report, rindex

## An Example

In [14]:
# read the data from an external csv file
macrodata = pd.read_csv('DSGEtestdata.csv')

# based on the data in the file calculate the natural logs of:
y = np.log(macrodata.RGDP)  # real GDP
c = np.log(macrodata.RCON)  # real consumption
i = np.log(macrodata.RINV)  # real inbestment
g = np.log(macrodata.RGOV)  # real government purchases
w = np.log(macrodata.NWAGE/macrodata.DEFL)  # real wage
e = np.log(macrodata.EMP)   # employment

# stack this data into a single numpy matrix
data = np.stack((y,c,i,g,w,e),axis=1)

# assign names for the columns
varnames = ['y','c','i','g','w','e']

# find number of observations and variables
(nobs, nvar) = data.shape

# choose a filter to remove the trend
filtertype = 'HP'

import statsmodels.api as sm
if filtertype == 'FD':   # first-differences
    cyclical = data[1:nobs,:] - data[0:nobs-1,:]
elif filtertype == 'HP': # Hodrick-Prescott
    cyclical, trend = sm.tsa.filters.hpfilter(data)
elif filtertype == 'CF': # Christiano-Fitzgerald
    cyclical, trend = sm.tsa.filters.cffilter(data)
elif filtertype == 'BK': # Baxter-King
    cyclical = sm.tsa.filters.bkfilter(data)

# use calcmom to get the default moments
moms, momnames = calcmom(cyclical)

# put into a pandas data frame
table = pd.DataFrame(moms, index = momnames, columns = varnames) 

# display on screen
from IPython.display import display, HTML
display(HTML(table.to_html()))

# write to an external csv file
table.to_csv('DSGEmoments.csv')

,y,c,i,g,w,e
standard deviations,0.016198,0.012534,0.074144,0.033072,0.018262,0.014307
standard deviations relative to GDP,1.000000,0.773792,4.577319,2.041723,1.127394,0.883242
corrleations with GDP,1.000000,0.770889,0.837286,0.155206,0.862408,0.820931
autocorrelations,0.850491,0.815801,0.796125,0.905775,0.876654,0.918816
